# Trabalho Prático 1

## Introdução

A resolução deste trabalho prático tem como objetivo implementar um abiente seguro, constituído por um emissor e um recetor, através de sessões cifradas e seguras.
A solução implmentada, foi estruturada da seguinte forma:

- Construção de uma sessão de comunicação síncrona entre um emissor e um recetor.
- Utilização do protocolo de **Diffie_Hellman** com verificação de chave e autenticação dos agentes, na sessão anteriormente referida.
- Utilização do **DSA** para a autenticação dos agentes.
- Utilização do **TAES** como cifra simétrica, com autenticação do criptograma em cada superbloco.
- Implementar novamente o cenário anterior mas, desta feita vez, utilizando o **ECDH** (Elliptic Curve Diffie-Hellman) em vez do **DH** e o **ECDSA** (Elliptic Curve Digital Signature Algorithm) em vez do **DSA**.

Este relatório está escrito de forma a que o texto escrito entre o código desenvolvido seja suficientemente explicativo do que está a ser implementado, o que, na nossa opinião, permite uma leitura e compreensão facilitada do mesmo.

## Código comum

Esta secção tem como objetivo definir _snippets_ de código que serão utilizados tanto na definição da sessão síncrona com **DH** como na sessão síncrona com **ECDH**, tais como os _imports_ de módulos necessários, as funções auxiliares necessárias e a classe de multiprocessamento.

### Imports

De seguida, encontram-se os módulos **Python** importados e necessários para desenvolver a sessão síncrona entre os agentes, utilizando, em primeira instância, o protocolo **DH** e o algoritmo de assinaturas **DSA** e, numa segunda instância, o protocolo **ECDH** e o algoritmo de assinaturas **ECDSA**.

In [1]:
import os,io
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.exceptions import *
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Definição das funções do processo criptográfico

Esta secção tem o objetivo de definir funções que serão utilizadas nas sessões síncronas, tanto na que utiliza **DH** como na que utiliza **ECDH**. Essas funções são as seguintes:

- `Hash(s)`, que tem como propósito calcular um _digest_ de uma dada mensagem.

- `next_tweak(initial,current_counter)`, que tem como propósito calcular o próximo _tweak_ a utilizar no bloco que vai ser cifrado/decifrado, com base no _nounce_ inicial e no número do bloco que está a ser processadoa atualmente.

- `bytes_xor(a,b)`, que tem como propósito realizar o _XOR_ _byte_ a _byte_ de duas dadas sequências de _bytes.

- `calculate_parity(blocks,number_blocks)`, que tem como objetivo calcular a paridade de um conjunto de blocos, para que seja gerada a sua _tag_ de autenticação.

- `calculate_auth_tweak(initial_tweak,number_blocks)`, que tem como objetivo calcular o _tweak_ utilizado para cifrar a paridade dos blocos cifrados.

- As funções `cifra_blocos(current_counter,initial_tweak,blocks,number_blocks,cipher_context)` e `decifra_blocos(current_counter,initial_tweak,blocks,number_blocks,decipher_context)` têm como objetivo cifrar/decifrar, respetivamente, um conjunto de blocos aplicando a cada bloco a função de cifra ou de decifragem.

- As funções `cifra(block,tweak,cipher_context)` e `decifra(block,tweak,decipher_context` têm como objetivo cifrar/decifrar um dado bloco utilizando para o efeito a função $ Ẽ(s,x) = E(s,x \oplus h(w)) \oplus h(w) $.

É relevante referir que as operações de cifra e de decifra utilizam exatamente a mesma fórmula e, logo, são iguais, mas estão separadas apenas por uma questão de mais simples compreensão do problema em questão, bem como da sua resolução.
Além disso, neste processo criptográfico foi utilizado o modo **TAE** (_Tweakable Authenticated Encryption_).

In [2]:
def Hash(s):
    digest = hashes.Hash(hashes.SHA256(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

def next_tweak(initial, current_counter):
    str = format(current_counter, '063b')
    str = str + '0'
    b_str = int(str,2).to_bytes((len(str) + 7) // 8, 'big')
    return (initial + b_str)

def bytes_xor(a, b) :
    return bytes(x ^ y for x, y in zip(a, b))

def calculate_parity(blocks,number_blocks):
    str = format(0,'128b') # O número 0 é elemento neutro no XOR
    parity = int(str,2).to_bytes((len(str) + 7) // 8, 'big')
    bytes_processed = 0
    while(bytes_processed < (16*number_blocks)):
        max_bytes = bytes_processed + 16
        block_to_xor = blocks[bytes_processed:max_bytes]
        parity = bytes_xor(parity,block_to_xor)
        bytes_processed += 16
    return parity

def calculate_auth_tweak(initial_tweak,number_blocks):
    str = format(number_blocks*16,'063b')
    str = str + '1'
    b_str = int(str,2).to_bytes((len(str) + 7) // 8, 'big')
    return (initial_tweak + b_str)

def cifra_blocos(current_counter,initial_tweak,blocks,number_blocks,cipher_context):
    bytes_processed = 0
    cryptogram = bytearray(160)
    while(bytes_processed < (16*number_blocks)):
        max_bytes = bytes_processed + 16
        block_to_process = blocks[bytes_processed:max_bytes]
        tweak = next_tweak(initial_tweak,current_counter)
        block_crypt = cifra(block_to_process,tweak,cipher_context)
        cryptogram[bytes_processed:max_bytes] = block_crypt
        bytes_processed += 16
        current_counter += 1
    return {"crypt": bytes(cryptogram), "i": current_counter}
        

def cifra(block,tweak,cipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,hw)
    e_s_x_hw = cipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram

def decifra_blocos(current_counter,initial_tweak,blocks,number_blocks,decipher_context):
    bytes_processed = 0
    text = bytearray(160)
    while(bytes_processed < (16*number_blocks)):
        max_bytes = bytes_processed + 16
        block_to_process = blocks[bytes_processed:max_bytes]
        tweak = next_tweak(initial_tweak,current_counter)
        block_text = decifra(block_to_process,tweak, decipher_context)
        text[bytes_processed:max_bytes] = block_text
        bytes_processed += 16
        current_counter += 1
    return {"text": bytes(text), "i": current_counter}

def decifra(block,tweak,decipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,hw)
    e_s_x_hw = decipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram
    

### Definição da classe de multiprocessamento

No seguinte código, é definida a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_, sendo estes dois processos criados e implementados pela **API** [**multiprocessing**](https://docs.python.org/2/library/multiprocessing.html).

In [3]:
class BiConnection(object):
    def __init__(self,left,right):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def auto(self,proc=None):
        if proc == None:
            self.left_process.start()
            self.right_process.start()
            self.left_process.join(self.timeout)
            self.right_process.join(self.timeout)
        else:
            proc.start()
            proc.join()
    
    def manual(self): 
        self.left()
        self.right()

## Sessão Síncrona com _Diffie-Hellman_

### Geração dos parâmetros _Diffie-Hellman_

A próxima secção implementa a geração dos parâmetros necessários para a derivação de chaves **DH** e **DSA**, por parte de ambos os agentes.

In [4]:
parameters_dh = dh.generate_parameters(generator=2,key_size=1024,backend=default_backend())
# O tamanho tem que ser 3072 bits, alterar depois.

### Geração das chaves permanentes dos agentes

Esta secção tem como objetivo gerar as chaves permanentes dos agentes envolvidos na comunicação, tendo em conta que estas são conhecidas por ambos, e serão aplicadas na assinatura digital das mensagens.

In [5]:
# Geração das chaves do Emissor
emitter_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
emitter_dsa_pk = emitter_dsa_sk.public_key() #chave pública DSA

# Geração das chaves do Recetor
receiver_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
receiver_dsa_pk = receiver_dsa_sk.public_key() #chave pública DSA

### Definição do Emissor e do Recetor

Por fim definiu-se o comportamento de cada um dos agentes nesta sessão síncrona segura de troca de informação. Na definição de ambos, é assumido que as chaves públicas **DSA** utilizadas na autenticação dos agentes já são conhecidas por ambos. As chaves **DH** são geradas por sessão de modo a acordar uma chave temporária, para que seja possível cifrar e decifrar uma mensagem utilizando uma primitiva simétrica.


In [6]:
def Emitter(connection):
    #Implementação Protocolo Diffie-Hellman
    emitter_dh_sk = parameters_dh.generate_private_key()
    emitter_dh_pk = emitter_dh_sk.public_key()
    pub = emitter_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = emitter_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk': pub, 'sig': signature}
    connection.send(message)
    receiver_first = connection.recv()
    receiver_dh_public_key = serialization.load_pem_public_key(receiver_first['pk'],default_backend())
    receiver_dsa_pk.verify(receiver_first['sig'],receiver_first['pk'],hashes.SHA256())
    master_key = emitter_dh_sk.exchange(receiver_dh_public_key)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    #Verificar a chave, enviando um hash da chave derivada para confirmar
    #que tanto emissor como recetor possuem a mesma chave.
    
    key_digest = Hash(shared_key)
    signature = emitter_dsa_sk.sign(key_digest,hashes.SHA256())
    connection.send({'digest': key_digest, 'sig': signature})
    
    key_check_obj = connection.recv()
    receiver_dsa_pk.verify(key_check_obj["sig"],bytes(key_check_obj["message"],'utf8'),hashes.SHA256())
    if key_check_obj["message"] == "OK":
        #Prosseguir com a cifra/decifra síncrona
        message_size = 1600
        gen_bytes = os.urandom(message_size)
        print('gen_bytes')
        print(gen_bytes)
        inputs = io.BytesIO(gen_bytes)
        #inicialização do tweak como um nounce de 64 bits (n/2)
        tweak_init = os.urandom(8)

        connection.send(tweak_init)

        buffer = bytearray(160)
        cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).encryptor()
        i = 1
        try:
            while inputs.readinto(buffer):
                cipher_object = cifra_blocos(i,tweak_init,buffer,10,cipher)
                cryptogram = cipher_object["crypt"]
                parity = calculate_parity(buffer,10)
                auth_tweak = calculate_auth_tweak(tweak_init,10)
                auth_tag = cifra(parity,auth_tweak,cipher)
                i = cipher_object["i"]
                msg_to_send = {'crypt': cryptogram, 'tag': auth_tag}
                connection.send(msg_to_send)
                msg = connection.recv()
                if(not(msg == 'OK')):
                    print('Erro no processo!')
                    break
        except Exception as err:
            print('Erro no emissor! {0}'.format(err))
        connection.send('finalized')   
        inputs.close()
    else:
        print('Chaves não coincidiram')
    connection.close()    

In [7]:
def Receiver(connection):
    #Implementação protocolo Diffie-Hellman
    emitter_first = connection.recv()
    receiver_dh_sk = parameters_dh.generate_private_key()
    receiver_dh_pk = receiver_dh_sk.public_key()
    emitter_dh_public_key = serialization.load_pem_public_key(emitter_first['pk'],default_backend())
    emitter_dsa_pk.verify(emitter_first['sig'],emitter_first['pk'],hashes.SHA256())
    master_key = receiver_dh_sk.exchange(emitter_dh_public_key)
    pub = receiver_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = receiver_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk':pub,'sig':signature}
    connection.send(message)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    key_check_obj = connection.recv()
    emitter_dsa_pk.verify(key_check_obj['sig'],key_check_obj['digest'],hashes.SHA256())
    digest = Hash(shared_key)
    if digest == key_check_obj["digest"]:
        message = "OK"
        signature = receiver_dsa_sk.sign(bytes(message,'utf8'),hashes.SHA256())
        obj = {"message":message, 'sig': signature}
        connection.send(obj)
        
        #prosseguir com o modo de decifra síncrono
    
        outputs = io.BytesIO()
        tweak_init = connection.recv()

        cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).decryptor()
        i = 1
        try:
            while True:
                buffer = connection.recv()
                if(buffer == 'finalized'):
                    outputs.write(cipher.finalize())
                    break
                else:
                    crypt = buffer['crypt']
                    tag = buffer['tag']
                    decipher_object = decifra_blocos(i, tweak_init,crypt,10,cipher)
                    text = decipher_object["text"]
                    i = decipher_object["i"]
                    parity = calculate_parity(text,10)
                    auth_tweak = calculate_auth_tweak(tweak_init,10)
                    auth_tag = decifra(tag,auth_tweak,cipher)
                    if(auth_tag == parity):
                        connection.send('OK')
                    else:
                        print('AUTH ERROR')
                        connection.send('AUTH ERROR')
                        break
                    outputs.write(text)
            print('decrypted:')
            print(outputs.getvalue())
        except Exception as Err:
            print('Erro no recetor! {0}'.format(Err))

        outputs.close()
    else:
        print('Chaves não coincidiram')
    connection.close()
    

In [8]:
BiConnection(Emitter,Receiver).auto()

gen_bytes
b'\x17\x19r\xbf\xce\xf4\xachZ\xb5:\xb0cQ\x1d\x1c\xa3\x12g\xdc\xe9lz\xec>\xc6\r\xe0\x04\xf2}pN\x1c\x13\xfef#^\xae\xfa\x06qf\x1c\x01)\xe3[\x86GY\x170\x8f\x9f\xf78y=ZC+\x02\x8e\x1d&\x1c\xbf}\xdfn\xd2\x07T\xedm\xa0\xb8\xd0"\xe7\xebqK\xe8wd\x80\xcb\xd3\xf8\x8eL\x12\x9f\xb3\x80 Y\xddm"\x1fs\x94LE(\xf0z^#\x9d\xf3\x96\x16\xa5;e\xa3\x9647u1\x9d\xed\x1c\x95\xd6u\xf1pZ\xff\xcc\xdf\x94I\xed\xec\x86\xc3\x99\xe1\x00\xfd\xed\xdf\x1e\xce\xf4\xd5\x89\xb7\x94iq\xb3\xcc"\xb0\x87\x9a~\x12}("\x16\xb3\xf6\x10\x86j\\\xe3\xc3\x93X*)\xaen\xd8\x88",\xf2^\xd8p\xa0\xa4\x8e\x12\x10\xae\x9f\xe1\xb4\x01\xc3K\xf1J\xac2\x81\x10\x03\xc9\x90\x006\x9a\x7f\xf59\xa8%\xb1\x08U\xb8\xbc\xa9\x99\x94\x01\x0bG\xeb\xac@$!\xb7P\x93\xcf\\)\xecQy\xda\xdbX\xa8T\xce\x10\x93<\xa4\xa8-8\xa3\xd7\xb8\xc6kE\xc87\xfc7(\x1d\xdb\x81Rnv\x918\xbfs\x16}\x17\xf3\xec\xb8\x16#\x98y\x19,\xbf\x977\x89\xf7X+%Vu|\x1d\xb6`r\x1f\x9b;6J,\xf4\x90g\x1d\xfa\x17U\xd0\xe9@p\x9b\x1d9\xd8\x03\xdc#UK\xd5\xa5\xc5\xbf\x9e\xc3\xad#\xdf\xea\x02\xff\xab\xe2\

A comparação entre os valores dados por **gen_bytes** e por **decrypted** no output, por via da igualdade, permitem concluir que a mensagem que foi gerada é a mesma mensagem que foi decifrada.

## Sessão Síncrona com _Elliptic Curve Diffie-Hellman_

### Geração das chaves permanentes dos agentes

Esta secção tem como objetivo gerar as chaves permanentes de ambos os agentes, ou seja, as chaves que são utilizadas na autenticação dos mesmos.

In [9]:
# Geração das chaves do emissor
emitter_ecdsa_sk = ec.generate_private_key(ec.SECP256R1(),default_backend()) #Chave privada
emitter_ecdsa_pk = emitter_ecdsa_sk.public_key() #Chave pública

# Geração das chaves do recetor
receiver_ecdsa_sk = ec.generate_private_key(ec.SECP256R1(),default_backend()) #Chave privada
receiver_ecdsa_pk = receiver_ecdsa_sk.public_key() #Chave pública

### Definição do Emissor e do Recetor

Nesta secção, é definido o comportamento de cada um dos agentes na sessão síncrona segura de troca de informação utilizando o protocolo **ECDH** e o algoritmo de assinatura **ECDSA**. É assumido que as chaves públicas **ECDSA** de cada agente já são conhecidas por ambos, pelo que podem ser utilizadas na autenticação dos agentes ao realizar o protocolo de acordo de chaves **ECDH**. As chaves **ECDH** são geradas por sessão, de forma a acordar uma chave de sessão temporária e comum que permita a utilização de uma primitiva simétrica.

In [10]:
def Emitter(connection):
    #Implementação do protocolo Elliptic Curve Diffie-Hellman
    emitter_ecdh_sk = ec.generate_private_key(ec.SECP521R1(), default_backend())
    emitter_ecdh_pk = emitter_ecdh_sk.public_key()
    pub = emitter_ecdh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo
    )
    signature = emitter_ecdsa_sk.sign(
        pub,
        ec.ECDSA(hashes.SHA256())
    )
    message_to_send = {'pub': pub, 'sig': signature}
    connection.send(message_to_send)
    receiver_info = connection.recv()
    receiver_ecdsa_pk.verify(receiver_info["sig"],receiver_info["pub"],ec.ECDSA(hashes.SHA256()))
    receiver_pub = serialization.load_pem_public_key(receiver_info["pub"],default_backend())
    master_key = emitter_ecdh_sk.exchange(ec.ECDH(),receiver_pub)
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    #Verificação da chave
    shared_key_digest = Hash(shared_key)
    signature = emitter_ecdsa_sk.sign(
        shared_key_digest,
        ec.ECDSA(hashes.SHA256())
    )
    msg = {'skd':shared_key_digest,'sig':signature}
    connection.send(msg)
    result = connection.recv()
    receiver_ecdsa_pk.verify(result['sig'],result['msg'],ec.ECDSA(hashes.SHA256()))
    if(result['msg'].decode('utf8') == 'OK'):
        # Iniciar processo de cifra
        message_size = 1600
        gen_bytes = os.urandom(message_size)
        print('bytes before cipher:')
        print(gen_bytes)
        inputs = io.BytesIO(gen_bytes)
        
        #Inicialização e envio dos primeiros 64 bits do tweak
        tweak_init = os.urandom(8)
        connection.send(tweak_init)
        
        buffer = bytearray(160)
        cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).encryptor()
        i = 1
        try:
            while inputs.readinto(buffer):
                cipher_object = cifra_blocos(i,tweak_init,buffer,10,cipher)
                cryptogram = cipher_object["crypt"]
                parity = calculate_parity(buffer,10)
                auth_tweak = calculate_auth_tweak(tweak_init,10)
                auth_tag = cifra(parity,auth_tweak,cipher)
                i = cipher_object["i"]
                msg_to_send = {'crypt': cryptogram, 'tag': auth_tag}
                connection.send(msg_to_send)
                msg = connection.recv()
                if(not(msg == 'OK')):
                    print('Erro no processo!')
                    break
        except Exception as err:
            print('Erro no emissor! {0}'.format(err))
        connection.send('finalized')   
        inputs.close()
    else:
        print(result['msg'].decode('utf8'))
    connection.close()
    

In [11]:
def Receiver(connection):
    #Implementação do protocolo Elliptic Curve Diffie-Hellman
    emitter_info = connection.recv()
    emitter_pub = emitter_info["pub"]
    sig = emitter_info["sig"]
    emitter_ecdsa_pk.verify(sig,emitter_pub,ec.ECDSA(hashes.SHA256()))
    receiver_ecdh_sk = ec.generate_private_key(ec.SECP521R1(),default_backend())
    receiver_ecdh_pk = receiver_ecdh_sk.public_key()
    pub = receiver_ecdh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo
    )
    signature = receiver_ecdsa_sk.sign(
        pub,
        ec.ECDSA(hashes.SHA256())
    )
    message_to_send = {'pub': pub, 'sig': signature}
    connection.send(message_to_send)
    key_check_info = connection.recv()
    emitter_ecdsa_pk.verify(key_check_info["sig"],key_check_info["skd"],ec.ECDSA(hashes.SHA256()))
    emitter_pub = serialization.load_pem_public_key(emitter_pub,default_backend())
    master_key = receiver_ecdh_sk.exchange(ec.ECDH(),emitter_pub)
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    #Verificação da chave
    shared_key_digest = Hash(shared_key)
    if(shared_key_digest == key_check_info["skd"]):
        message = bytes("OK",'utf8')
        signature = receiver_ecdsa_sk.sign(
            message,
            ec.ECDSA(hashes.SHA256())
        )
        connection.send({'msg':message, 'sig': signature})
        # Iniciar processo de decifra
        tweak_init = connection.recv()
        outputs = io.BytesIO()
        
        cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).decryptor()
        i = 1
        try:
            while True:
                buffer = connection.recv()
                if(buffer == 'finalized'):
                    outputs.write(cipher.finalize())
                    break
                else:
                    crypt = buffer['crypt']
                    tag = buffer['tag']
                    decipher_object = decifra_blocos(i, tweak_init,crypt,10,cipher)
                    text = decipher_object["text"]
                    i = decipher_object["i"]
                    parity = calculate_parity(text,10)
                    auth_tweak = calculate_auth_tweak(tweak_init,10)
                    auth_tag = decifra(tag,auth_tweak,cipher)
                    if(auth_tag == parity):
                        connection.send('OK')
                    else:
                        print('AUTH ERROR')
                        connection.send('AUTH ERROR')
                        break
                    outputs.write(text)
            print('decrypted:')
            print(outputs.getvalue())
        except Exception as Err:
            print('Erro no recetor! {0}'.format(Err))

        outputs.close()
    else:
        message = bytes("ERROR IN KEY CHECK",'utf8')
        signature = receiver_ecdsa_sk.sign(
            message,
            ec.ECDSA(hashes.SHA256())
        )
        connection.send({'msg':message, 'sig': signature})
    connection.close()
    

In [12]:
BiConnection(Emitter,Receiver).auto()

bytes before cipher:
b'\'\xe1\xa6\xa9\xfcK(\xdc\x1e\x05\x8a\x92&\xff\xd5y3VH\xcb\xe5\x85\xe5\xab\xa9\xba\xb2C^\x8crEo\xd6\xb1\xfd\x80\x88\x92\xbc\xfd\x10sbX\x12f)\xc0Y7\xc6^\xbf\x08\x0c\x8f\xc7\x9c\x1dS\xc1\x91=X\xfce\xf9\xddA\xbf\x81\x98@u\xc4P\x0f\xde\x07\xacY\xfa\xb3\x94\x8a\xe5]/\x03\xb0\xd4R\xbb3h\x07\xd3s:\xf2\xfb^!\x9e\xc7#\x8aa\xbep\xf1\x9b\xfbi<\xa6\xa4b_Xs\x8b\xca\x86\x8d\x9e\xaam\xcfM\xd5\xf4\xaeL\xe0`V\xe8\xc6\xd2\x08\x07\xdfH-\x12\x13\x8c\xb8\x8cD6\x8b\xfbX\xda\x9e\x03b\xc2d\xef\xeceyCy]\x002S\xef\x94\x129\xfe\xbc\x8f\xd7Y\xa8U\xb3\xf0\x93g\xcb\xe3\xd4\xa4\x07\xf7\x9a:\xea\x94\xce\x8c:\xaf\x02-\xe5\xa9\x7f\xa1c\xb6\x9f\xcb\xa9+uM\xd8\xdc\xb2U\x89\xae\xf7\xf9\xb1\x82\xa1n\x1cW\xea\xb3\xe1~\x03\xbb\xb4\x93\xf5\x10Wy\xb5\xffn{?\xb0\x89\xadA\x86\x88\x9138g\x8f\xba.\xda\xda\xfc\x12sZ\xf2:o\x07\xc5\xd2\xff\x90\xb4$\xe8V\x17\xc5\xd9\xa7f\x89}\x04[\x1a\xd7\xc1\xde\x1a|<<\xf6\xd5\xd7}9\x9d\xcf=\xb1e\xacS\xf1n\xad\xbb\xb0x\n\x927\x06\x9d\xf8\xd02\xe6z\x1bf\x95\xa4\xf9\x98\x11Gr`\xc6

A comparação entre os valores dados por **bytes before cipher** e por **decrypted** no output, por via da igualdade, permitem concluir que a mensagem que foi gerada é a mesma mensagem que foi decifrada.

## Conclusão

Os resultados da realização deste trabalho prático são, na nossa opinião, muito satisfatórios visto que fomos capazes de cumprir com todos os objetivos propostos ao implementar duas sessões síncronas seguras de troca de informação entre agentes, uma utilizando o protocolo **DH** e o algoritmo de assinaturas **DSA** e outra utilizando o protocolo **ECDH** e o algoritmo de assinaturas **ECDSA**. Além disso, a apresentação dos resultados (código e explicação) está feita de tal forma que seja mais simples entender o que foi a estratégia do grupo para a resolução dos problemas propostos.

No processo de resolução deste trabalho prático, o grupo deparou-se apenas com duas dificuldades que foram, eventualmente, ultrapassadas com mais ou menos esforço. Uma delas diz respeito à implementação do **TAES** devido ao facto de a primitiva ter que ser efetivamente implementada antes de a utilizar o que, no início, gerou um bocado de confusão em relação ao que devia ser feito. A segunda dificuldade esteve relacionada com o facto da especificação da sessão síncrona, visto que, esta foi a primeira vez que o grupo implementou algo do género, pelo que as características deste tipo de sessão tiveram que ser bem estabelecidades e diferenciadas em relação ao tipo de sessão que estávamos habituados a implementar (sessão assíncrona).

## Referências

1. [Worksheets TP2 fornecidas pelo professor](https://www.dropbox.com/sh/f0j9adiaw4v3deb/AADIuhV5bL5qqzbM_RLS-gMXa/WorkSheets/TP2?dl=0&subfolder_nav_tracking=1)
2. [Cryptography - Elliptic Curve cryptography](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ec/#)
3. [Cryptography - Diffie-Hellman Key Exchange](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/dh/)
4. [Tweakable Block Ciphers - paper by Liskov, Rivest and Wagner](https://people.eecs.berkeley.edu/~daw/papers/tweak-crypto02.pdf)
5. [Tweakable Block Ciphers - Notas manuscritas do professor](https://www.dropbox.com/sh/f0j9adiaw4v3deb/AAC5XUPllha9QXo-vVxTHqXna/Docs/NOTAS_MANUSCRITAS?dl=0&preview=TWEAKABLE_BLOCK_CIPHERS.PDF&subfolder_nav_tracking=1)